In [2]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
import requests

SYSTEM_PROMPT = f"""당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.
실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)
추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.
당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.
"""

MESSAGES = [
    {
        'role': 'system',
        'content': SYSTEM_PROMPT
    }
]

def recommend(query_text):
    url = "http://localhost:8000/recommend"
    response = requests.post(url, json={"query_text": query_text})
    return response.json()


tools = [
    {
        "type": "function",
        "function": {
            "name": "recommend",
            "description": "사용자 발화 기반으로 메뉴 추천 API를 호출합니다. 오로지 이 함수 결과로만 메뉴 추천되어야 합니다. 사용자 발화가 없는 경우 빈 리스트가 반환 될 수 있습니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_text": {
                        "type": "string",
                        "description": "사용자 발화 텍스트 원문",
                    },
                },
                "required": ["query_text"],
                "additionalProperties": False,
            },
        }
    }
]

In [4]:
MESSAGES.append(
    {"role": "user", "content": "안녕"}
)

completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)

completion

ChatCompletion(id='chatcmpl-BScp4tPjx6UD47G5FYVKodoXhFBjW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요! 무엇을 도와드릴까요? 해장이 필요하시거나 다이어트 중이신가요? 필요하신 메뉴가 있다면 말씀해 주세요! 적절한 메뉴를 추천해드리겠습니다.', refusal=None, role='assistant', function_call=None, tool_calls=None, annotations=[]))], created=1746162118, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_90122d973c', usage=CompletionUsage(completion_tokens=49, prompt_tokens=244, total_tokens=293, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))

In [5]:
if completion.choices[0].finish_reason == "tool_calls":
    print('function calling lets go')
else:
    MESSAGES.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )

In [6]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 무엇을 도와드릴까요? 해장이 필요하시거나 다이어트 중이신가요? 필요하신 메뉴가 있다면 말씀해 주세요! 적절한 메뉴를 추천해드리겠습니다.'}]

In [7]:
MESSAGES.append(
    {"role": "user", "content": "해장 메뉴 추천해줄레?"}
)

completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)

completion

ChatCompletion(id='chatcmpl-BScpKLRSAm90zVTlmmivUYuCaZKvj', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_eM0jjthCtKQdQJKVHxTGzNZe', function=Function(arguments='{"query_text":"해장"}', name='recommend'), type='function')], annotations=[]))], created=1746162134, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_90122d973c', usage=CompletionUsage(completion_tokens=17, prompt_tokens=307, total_tokens=324, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))

In [8]:
import json

if completion.choices[0].finish_reason == "tool_calls":
    tool_calls = completion.choices[0].message.tool_calls
    tool_name = tool_calls[0].function.name
    tool_args = tool_calls[0].function.arguments
    tool_id = tool_calls[0].id
    print('function calling lets go')
else:
    MESSAGES.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )

function calling lets go


In [9]:
completion.choices[0].message.tool_calls[0]

ChatCompletionMessageToolCall(id='call_eM0jjthCtKQdQJKVHxTGzNZe', function=Function(arguments='{"query_text":"해장"}', name='recommend'), type='function')

In [10]:
tool_result = recommend(**json.loads(tool_args))
tool_result[:3]

[{'_id': '1133486_26',
  'score': 0.5811076300414256,
  'menu': '우동',
  'restaurant': '중화요리현',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1133486/'},
 {'_id': '1015006_23',
  'score': 0.5307030745275177,
  'menu': '짬뽕',
  'restaurant': '아성원',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1015006/'},
 {'_id': '25883_12',
  'score': 0.5116151078788386,
  'menu': '',
  'restaurant': '중국성(가산동)',
  'url': 'https://www.yogiyo.co.kr/mobile/#/25883/'}]

In [11]:
tool_calls[0].id

'call_eM0jjthCtKQdQJKVHxTGzNZe'

In [12]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 무엇을 도와드릴까요? 해장이 필요하시거나 다이어트 중이신가요? 필요하신 메뉴가 있다면 말씀해 주세요! 적절한 메뉴를 추천해드리겠습니다.'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'}]

In [13]:
MESSAGES.append({
    "role": "assistant",  # user does not work
    "content": None,
    "tool_calls": [
        {
            "id": tool_id,
            "type": "function",
            "function": {
                "name": tool_name,
                "arguments": tool_args
            }
        }
    ]
})

In [14]:
tool_args

'{"query_text":"해장"}'

In [15]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 무엇을 도와드릴까요? 해장이 필요하시거나 다이어트 중이신가요? 필요하신 메뉴가 있다면 말씀해 주세요! 적절한 메뉴를 추천해드리겠습니다.'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_eM0jjthCtKQdQJKVHxTGzNZe',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"해장"}'}}]}]

In [17]:
tool_result[:5]

[{'_id': '1133486_26',
  'score': 0.5811076300414256,
  'menu': '우동',
  'restaurant': '중화요리현',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1133486/'},
 {'_id': '1015006_23',
  'score': 0.5307030745275177,
  'menu': '짬뽕',
  'restaurant': '아성원',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1015006/'},
 {'_id': '25883_12',
  'score': 0.5116151078788386,
  'menu': '',
  'restaurant': '중국성(가산동)',
  'url': 'https://www.yogiyo.co.kr/mobile/#/25883/'},
 {'_id': '25883_11',
  'score': 0.5069410369242237,
  'menu': '짬뽕밥',
  'restaurant': '중국성(가산동)',
  'url': 'https://www.yogiyo.co.kr/mobile/#/25883/'},
 {'_id': '1359676_14',
  'score': 0.49885010640655647,
  'menu': '황치즈',
  'restaurant': '옐로우캔-독산직영점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1359676/'}]

In [18]:
MESSAGES.append({
    "role": "tool",
    "content": json.dumps(tool_result[:3], ensure_ascii=False),
    "tool_call_id": tool_id
})

In [19]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 무엇을 도와드릴까요? 해장이 필요하시거나 다이어트 중이신가요? 필요하신 메뉴가 있다면 말씀해 주세요! 적절한 메뉴를 추천해드리겠습니다.'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_eM0jjthCtKQdQJKVHxTGzNZe',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"해장"}'}}]},
 {'role': 'tool',
  'content': '[{"_id": "1133486_26", "score": 0.5811076300414256, "menu": "우동", "restaurant": "중화요리현", "url": "https://www.yogiyo.co.kr/mobile/#/1133486/"}, {"_id": "1015006_23", "score": 0.5307030745275177, "menu": "짬뽕", "restaurant": "아성원", "url": "https:/

In [20]:
completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)
completion

ChatCompletion(id='chatcmpl-BScqO9pipPFaS8tm1Kh0PkP3PytVV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='해장을 위해 추천드리는 메뉴입니다:\n\n1. **우동** - [중화요리현](https://www.yogiyo.co.kr/mobile/#/1133486/)에서 주문 가능합니다.\n2. **짬뽕** - [아성원](https://www.yogiyo.co.kr/mobile/#/1015006/)에서 즐기실 수 있어요.\n3. 다양한 중화 요리를 위해 [중국성(가산동)](https://www.yogiyo.co.kr/mobile/#/25883/)을 방문해보세요.\n\n맛있는 식사 되세요! 필요하신 게 더 있으시면 말씀해 주세요.', refusal=None, role='assistant', function_call=None, tool_calls=None, annotations=[]))], created=1746162200, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_90122d973c', usage=CompletionUsage(completion_tokens=138, prompt_tokens=513, total_tokens=651, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}))

In [21]:
MESSAGES.append(
    {"role": "assistant", "content": completion.choices[0].message.content}
)

In [22]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 무엇을 도와드릴까요? 해장이 필요하시거나 다이어트 중이신가요? 필요하신 메뉴가 있다면 말씀해 주세요! 적절한 메뉴를 추천해드리겠습니다.'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄레?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_eM0jjthCtKQdQJKVHxTGzNZe',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"해장"}'}}]},
 {'role': 'tool',
  'content': '[{"_id": "1133486_26", "score": 0.5811076300414256, "menu": "우동", "restaurant": "중화요리현", "url": "https://www.yogiyo.co.kr/mobile/#/1133486/"}, {"_id": "1015006_23", "score": 0.5307030745275177, "menu": "짬뽕", "restaurant": "아성원", "url": "https:/